## Clean and standardise column names
We want consistent, lower_snake_case column names across the pipeline.

# ⚙️ 02 – Feature Engineering & Label Definition

**Objective:**  
Transform the ingested Netflix dataset into a clean, structured, and model-ready format by standardising column names, deriving new features, and defining the target label (`is_hit`).

**Scope of this notebook:**
- Load the raw data from the Delta table **`netflix_raw`** created in Notebook 01  
- Standardise all column names to `snake_case` for consistency  
- Parse and transform key fields:
  - Extract `release_year` from `release_date`
  - Derive numeric `duration_num` from `duration`
  - Add a binary flag `is_movie` based on category  
- Handle missing or null values where appropriate  
- Define a preliminary target label `is_hit` to support downstream modelling  
- Save the cleaned, feature-rich dataset as **`netflix_clean`** (or **`netflix_model_data`**) for use in Notebook 03

**Outcome:**  
A fully prepared dataset suitable for exploratory modelling, feature selection, and predictive analysis within Databricks.

In [0]:
df_raw = spark.table("netflix_raw")

In [0]:
import re

def to_snake_case(name):
    name = re.sub(r'[^0-9a-zA-Z]+', '_', name)
    name = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.lower().strip('_')

df_clean = df_raw.select([F.col(c).alias(to_snake_case(c)) for c in df_raw.columns])

display(df_clean)

In [0]:
df_clean = df_raw.select([F.col(c).alias(to_snake_case(c)) for c in df_raw.columns])

In [0]:
df_clean = (
    df_clean
    .withColumn("release_year", F.year(F.to_date("release_date", "MMMM d, yyyy")))
    .withColumn("duration_num", F.regexp_extract("duration", r"(\d+)", 1).cast("int"))
    .withColumn("is_movie", F.when(F.lower("category") == "movie", 1).otherwise(0))
)

## Parse and clean key fields
Extract the release year and normalise the duration field (minutes vs seasons).

In [0]:
from pyspark.sql import functions as F

df_clean = (
    df_clean
    # Extract year from release_date (handles “September 9, 2019” format)
    .withColumn("release_year", F.year(F.to_date(F.col("release_date"), "MMMM d, yyyy")))
    # Extract numeric duration (e.g. “90 min” → 90, “2 Seasons” → 2)
    .withColumn(
        "duration_num",
        F.regexp_extract(F.col("duration"), r"(\d+)", 1).cast("int")
    )
    # Label whether it’s a Movie or TV Show
    .withColumn(
        "is_movie",
        F.when(F.lower(F.col("category")) == "movie", 1).otherwise(0)
    )
)

## Define a basic “hit” label (is_hit)
The dataset doesn’t have view counts or IMDb scores, so let’s create a proxy “popularity” label using the text length of the description — purely for demonstration. Later, if we merge external IMDb data, we’ll replace it with real metrics.

In [0]:
# Heuristic hit label based on description length (as a placeholder)
df_clean = df_clean.withColumn(
    "is_hit",
    F.when(F.length(F.col("description")) > 120, 1).otherwise(0)
)

## Save as a Delta table
This becomes our modelling dataset for the next notebook.

In [0]:
(
    df_clean.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("netflix_clean")
)

print("Saved table: netflix_clean")

# Quick preview
display(spark.table("netflix_clean").select("title", "category", "release_year", "duration_num", "is_hit").limit(10))

## 🧾 Notebook Summary – 02_Feature_Engineering_And_Label

**Objective:**  
Transform the raw Netflix dataset (`netflix_raw`) into a clean, feature-rich, and model-ready format for machine learning.

**Key steps completed:**
- Loaded data from the ingested Delta table **`netflix_raw`**
- Standardised all column names to `snake_case` using a reusable Python function  
- Parsed and engineered key fields:
  - Extracted `release_year` from `release_date`
  - Converted `duration` to numeric `duration_num`
  - Added binary flag `is_movie` for category type
- Created a preliminary `is_hit` label (placeholder for future popularity metric integration)
- Persisted the transformed dataset as a managed Delta table: **`netflix_clean`**

**Findings:**
- The dataset is now structured and ready for feature selection and modelling
- Most categorical features (`rating`, `category`, `country`) contain useful variance for prediction
- Nulls remain in descriptive fields (e.g., `director`, `cast`) but do not block training

**Next steps:**
- Proceed to **`03_modelling_and_evaluation`** to:
  - Train a baseline classifier (e.g., Random Forest)
  - Evaluate model performance using ROC-AUC, precision, and recall
  - Track experiments in MLflow
  - Generate insights and feature importance visualisations